In [1]:
import os 
import imp
import warnings
import numpy as np
import pandas as pd
import importlib as imp
import itertools
from tqdm import tqdm, tqdm_notebook
from utils import tools
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

PATH_TO_BENCHMARKS = './/data//'

**Задача**
$asd$
*Ваша четвертая лабораторная работа будет заключаться в реализации одного из двух алгоритмов: <br>
метод имитации обжига или генетический алгоритм со встроенной эвристикой в виде локального поиска.  <br>
Алгоритмы реализуются для задачи Cell formation problem.* <br>

Реализовать можно на любом языке. <br>
Также нужно будет сравнить время работы алгоритма, а так же показать лучшее решение алгоритма, которое удалось найти вам. <br>

**Данные:**

Данные для работы прикрепил архивом к письму. **Структура такая:** <br>

m p (number of machines and parts) <br>
Next m rows: <br>
m(row number) list of parts processed by machine m separated by space <br>
e.g: <br>
1 9 17 19 31 33
means machine 1 processes parts 9 17 19 31 33 <br>

**Выходной формат для ответов:** <br>
Output file: instancename.sol (e.g. 20x20.sol) <br>
Output file format: <br>
m1_clusterId m2_clusterId ... - machines to clusters mapping <br>
p1_clusterId p2_clusterId ... - parts to clusters mapping <br>

In [2]:
txtfiles = [f for f in os.listdir(PATH_TO_BENCHMARKS) if os.path.isfile(os.path.join(PATH_TO_BENCHMARKS, f))]
txtfiles

['20x20.txt', '24x40.txt', '30x50.txt', '30x90.txt', '37x53.txt']

In [12]:
from algorithm import SimulatedAnnealing
from datetime import datetime
tools = imp.reload(tools)

SimulatedAnnealing = imp.reload(SimulatedAnnealing)
params = {
    'initial_temperature' : 10,
    'final_temperature'   : 0.002,
    'chain_len'           : 4,
    'len_of_period'       : 6,
    'numb_of_cells'       : 2,
    'check'               : 5,
    'cooling_rate'        : 0.7,
}

for txt in txtfiles:
    print(txt)
    machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txt)
    cells_p, cells_m = tools.get_solution(machine_part_matrix, 2)

    SA = SimulatedAnnealing.SimulatedAnnealing(machine_part_matrix)
    SA.set_params(params)
    start = datetime.now()
    SA.solve()
    print('time:', datetime.now() - start)

20x20.txt
              f: {'current': 0.3053097345132743, 'neigh': 0, 'best': 0}
temperature    : {'current': 10, 'final': 0.002}
number of cells: {'initial': 2, 'optimal': 0} cool_rate: 0.7


              f: {'current': 0.32903225806451614, 'neigh': 0.32903225806451614, 'best': 0.3973509933774834}
temperature    : {'current': 0.0019158123138056616, 'final': 0.002}
number of cells: {'initial': 6, 'optimal': 5} cool_rate: 0.7


time: 0:00:26.784682
24x40.txt
              f: {'current': 0.1790744466800805, 'neigh': 0, 'best': 0}
temperature    : {'current': 10, 'final': 0.002}
number of cells: {'initial': 2, 'optimal': 0} cool_rate: 0.7


              f: {'current': 0.22321428571428573, 'neigh': 0.22321428571428573, 'best': 0.26881720430107525}
temperature    : {'current': 0.0019158123138056616, 'final': 0.002}
number of cells: {'initial': 5, 'optimal': 4} cool_rate: 0.7


time: 0:01:36.037527
30x50.txt
              f: {'current': 0.10413476263399694, 'neigh': 0, 'best': 0}
temperat